# Check Patient Population

In [ ]:
import mimic_pipeline as mmp
import mimic_pipeline.utils as utils

In [ ]:
user = input("Enter your username: ")
password = input("Enter your password: ")
load = utils.DataBaseLoader(user=user, password=password)
data = load['union_features_prep']
data.head()

Check urineoutput

In [ ]:
print(len(data[data['urineoutput'] < 0]))
data['urineoutput'].describe()

Check Pre-ICU length of stay

In [ ]:
print(len(data[data['preiculos'] < 0]))
data['preiculos'].describe()


check age

In [ ]:
data['age'].describe()

check uniqueness

In [ ]:
data['subject_id'].is_unique

In [ ]:
len(data['subject_id'])

check ICU length of stay

In [ ]:
import pandas as pd
icustay_df = load['icustays']
data_check = pd.merge(data, icustay_df, on=['subject_id', 'hadm_id', 'icustay_id'])
data_check['los'].describe()

check no burns

In [ ]:
diagnoses_icd = load['diagnoses_icd']
diagnoses_icd = diagnoses_icd[(diagnoses_icd['icd9_code'] >= '940') & (diagnoses_icd['icd9_code'] <= '946')]

In [ ]:
assert not data['subject_id'].isin(diagnoses_icd['subject_id']).all()       # check burn patients are not in the data

check hospital stay length

In [ ]:
adm = load['admissions']
data_adm = pd.merge(data, adm, on=['subject_id', 'hadm_id'], how='left')
data_adm['hosp_los'] = (data_adm['dischtime'] - data_adm['admittime']).dt.total_seconds() / 86400

In [ ]:
len(data_adm[data_adm['hosp_los'] < 0])

In [ ]:
data_adm['hosp_los'].describe()

check first ICU stay

In [ ]:
icustay_df = load['icustays']
icustay_df = icustay_df.sort_values('intime')
first_stay_df = icustay_df.groupby('subject_id').first().reset_index()
assert data['subject_id'].isin(first_stay_df['subject_id']).all()       # check all patients are first stay patients
assert data['icustay_id'].isin(first_stay_df['icustay_id']).all()       # check all icustays are first icustays
assert data['hadm_id'].isin(first_stay_df['hadm_id']).all()             # check all hadms are first hadms